In [2]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

import plotly
import plotly.graph_objs as go
import plotly.io as pio

In [3]:
# Load dataframes from csv files
census_df = pd.read_csv('df_census.csv', index_col=0)
ec_df = pd.read_csv('df_ec.csv', index_col=0)
election_df = pd.read_csv('df_election.csv', index_col=0)
abbrev_df = pd.read_csv('df_abbrev.csv', index_col=0)

In [4]:
print(census_df.loc["United States", census_df.columns[0]])
print(sum(census_df.loc["Alabama":"Wyoming", "HC01_VC03"]))
print(type(census_df.loc["Alabama":"Wyoming", "HC01_VC03"]))
census_df.loc[:, "HC01_VC03"].dtype
ec_df["Number of Electoral Votes"].dtype

325719178
325719178
<class 'pandas.core.series.Series'>


dtype('int64')

In [5]:
# Calculate Average voting power (EV share PER person)
avg_vp = ec_df["Number of Electoral Votes"].sum() / float(census_df.loc["United States", "HC01_VC03"])
print('Average share of an electoral vote for an individual:\n{:.2e}'
      .format(avg_vp))

# Store inverse
people_per_ev = 1.0/avg_vp
print('\nAverage amount of people per electoral vote:\n{:.2f}'.format(people_per_ev))

Average share of an electoral vote for an individual:
1.65e-06

Average amount of people per electoral vote:
605425.98


### I. Electoral Vote Apportionment Bar Chart

In [6]:
# Get list of states in decreasing order of population
states = sorted(list(census_df.index[1:]), 
                key=lambda state: census_df.loc[state, 'HC01_VC03'],
                reverse=True)

state_pop = go.Bar(
    x=list(range(51)),#states,
    y=[census_df.loc[state, 'HC01_VC03'] for state in states],
    name='Population',
    marker=dict(
        color='#C2EFEB',
        #opacity=,
    )
)

ev_color = '#6EA4BF'
state_ev = go.Bar(
    x=list(range(51)),#states,
    y=[ec_df.loc[state, 'Number of Electoral Votes'] for state in states],
    name='Electoral Votes',
    yaxis='y2',
    #offset=4,
    marker=dict(
        color=ev_color,
        #opacity=,
    ),
    width=0.2,
)

# Hacky way to get grouped bars on multiple y-axes. Not working.
dummies = [
    go.Bar(
        x=['Alabama'], 
        y=[0],
        yaxis='y',
        name='y dummy', 
        hoverinfo='none', 
        showlegend=False
    ), 
    go.Bar(
        x=['Alabama'],
        y=[0],
        yaxis='y2', 
        name='y2 dummy', 
        hoverinfo='none', 
        showlegend=False
    ),
]

b_top = census_df.loc['California', 'HC01_VC03']
b_bot = 33300000
b_x = 1.2
b_size = 1.0
bracket = go.Scatter(
    x=[b_x, b_x+b_size, b_x+b_size, b_x],
    y=[b_bot, b_bot, b_top, b_top],
    fill=None,
    mode='lines',
    line=dict(
        width=1.0,
        color='black',
    ),
)

data = [state_pop, state_ev] + [bracket] #+ dummies

layout = go.Layout(
    title=dict(   
        text='State populations and electoral votes are NOT proportional',
        font=dict(
            family='verdana',
            size=18,
            color=None,
        ),
        #xref='paper',
        yref='paper',
        y=1.0,
        yanchor='bottom',
        #xanchor='right',
        pad=dict( # unit: pixels
            b=20,
        ),
    ),
    width=900,
    height=500,
    #barmode='group',
    showlegend=False,
    xaxis=dict(
        tickfont=dict(
            size=10,
        ),
        tickangle=-65,
        #automargin=True,
        tickmode='array',
        tickvals=list(range(51)),
        ticktext=states[:states.index('District of Columbia')] \
                 + ['D.C.'] + states[states.index('District of Columbia')+1:] 
    ),
    yaxis=dict(
        title='Population',
        range=[0,40000000],
        tickfont=dict(
            size=12,
        ),
        rangemode='tozero',
    ),
    yaxis2=dict(
        title='Electoral Votes',
        range=[0,40000000/float(people_per_ev)],
        color=ev_color,
#         titlefont=dict(
#             color=ev_color,
#         ),
        tickfont=dict(
            size=12,
        ),
        overlaying='y',
        side='right',
        rangemode='tozero',
        zeroline=False,
    )
)

notes = [dict(
    text='<i>This gap represents population<br>' +\
         'not represented by electoral votes<br>' +\
         '<b>More people than votes!</b></i>',
    font=dict(
        family=None,
        size=12,
        color='black',
    ),
    x=b_x + b_size,
    y=(b_top + b_bot)/2.0,
    xref='x',
    yref='y',
    align='left',
    xanchor='left',
    yanchor='middle',
    showarrow=True,
    arrowcolor='black',
    arrowwidth=1.2,
    arrowhead=0,
    arrowsize=1,
    startstandoff=3,
    axref='x',
    ayref='y',
    ax=6,
    ay=(b_top + b_bot)/2.0,
    xshift=0,
)]

layout.update(annotations=notes)

fig = go.Figure(data=data, layout=layout)
plotly.offline.init_notebook_mode(connected=True)
plotly.offline.iplot(fig)

### II. US Map by Voting Power

In [8]:
# Try out function to convert matplotlib colormaps to Plotly format
import matplotlib
from matplotlib import cm
import numpy as np

magma_cmap = matplotlib.cm.get_cmap('PiYG_r')

def matplotlib_to_plotly(cmap, pl_entries):
    h = 1.0/(pl_entries-1)
    pl_colorscale = []

    for k in range(pl_entries):
        C = list(map(np.uint8, np.array(cmap(k*h)[:3])*255))
        pl_colorscale.append([k*h, 'rgb'+str((C[0], C[1], C[2]))])

    return pl_colorscale

magma = matplotlib_to_plotly(magma_cmap, 255)
#magma[123:132]
modm = magma[:123] + [[0.49, 'rgb(255, 255, 255)'],[0.51, 'rgb(255, 255, 255)']] + magma[132:]
modm[120:126]

[[0.47244094488188976, 'rgb(242, 246, 235)'],
 [0.4763779527559055, 'rgb(242, 246, 237)'],
 [0.48031496062992124, 'rgb(243, 246, 238)'],
 [0.49, 'rgb(255, 255, 255)'],
 [0.51, 'rgb(255, 255, 255)'],
 [0.5196850393700787, 'rgb(248, 242, 245)']]

In [145]:
norm_states_vp[states.index('District of Columbia')]

2.6172207873486513

In [170]:
# Calculate State Voting Power (SVP) and normalize by average voting power
states_vp = [ec_df.loc[state, "Number of Electoral Votes"] / \
             float(census_df.loc[state, "HC01_VC03"]) for state in states]
norm_states_vp = [vp/avg_vp for vp in states_vp]

rasp_green = [
    [0.0, 'rgb(144,11,85)'],
    #[0.4, 'blue'],
    [0.495, 'white'],
    [0.505, 'white'],
    [0.6, 'rgb(43,102,30)'],
    [1.0, 'rgb(43,102,30)']
]

orange_green = [
    [0.0, 'rgb(130,61,17)'],
    [0.25, 'rgb(218,126,38)'],
    [0.495, 'white'],
    [0.505, 'white'],
    [0.7, 'rgb(4,62,50)'],
    [1.0, 'rgb(4,62,50)']
]

# Diverging scheme from colorbrewer2.org
Pi = 'rgb(208,28, 139)'
G = 'rgb(77,172,38)'
PiYG = [
    [0.0, Pi],
    [0.25, 'rgb(241,182,218)'],
    #[0.495, 'white'],
    [0.50, 'white'],
    [0.52, 'rgb(184,225,134)'],
    [0.55, G],
    [1.0, G]
]

# Diverging scheme from colorbrewer2.org
Or = 'rgb(230,97,1)' #'rgb(253,184,99)'#
BG = 'rgb(53,151,143)' #'rgb(90,180,172)' #

OrBG = [
    [0.0, Or],
    [0.25, 'rgb(253, 184, 99)'],
    [0.50, 'white'],
    [0.52, 'rgb(128,205,193)'],
    [0.55, BG],
    [1.0, BG]
]

# Plot
vp_map = go.Choropleth(
    colorscale=magma, #OrBG, #PiYG, #rasp_green,#'RdBu',
    autocolorscale=False,
    reversescale=True,
    locations=[abbrev_df.loc[state, 'Postal Code'] for state in states], # Needs abbreviations
    z=norm_states_vp,
    locationmode='USA-states',
    marker=dict(
        line=dict(
            color='black',
            width=0.2,
        )
    ),
    zauto=True,
    zmid=1.0,
#     zmin=min(norm_states_vp),   # does nothing while auto=True
#     zmax=max(norm_states_vp),
    colorbar = dict(
        title = ""
    )
)

vp_map2 = go.Choropleth(
    colorscale=PiYG, #rasp_green,#'RdBu',
    autocolorscale=False,
    reversescale=True,
    locations=['WY'], # Needs abbreviations
    z=[3.1],
    locationmode='USA-states',
    marker=dict(
        line=dict(
            color='black',
            width=0.2,
        )
    ),
    zauto=True,
    zmid=1.0,
#     zmin=min(norm_states_vp),   # does nothing while auto=True
#     zmax=max(norm_states_vp),
#     colorbar = dict(
#         title = ""
#     )
)

dc_overlay = go.Scattergeo(
        locations=['DC'],
        locationmode='USA-states',
        name='',
        mode='markers',
        opacity=1.0,
        marker=dict(
            symbol='circle',
            size=10,
            line=dict(
                color='black',
                width=0.2,
            ),
            color=[1.0],#norm_states_vp[states.index('District of Columbia')]],
            colorscale=OrBG,
            reversescale=True,
            cauto=False,
            cmid=1.0,
#             cmin=min(norm_states_vp),
            cmax=max(norm_states_vp),
            showscale=True,
            colorbar=dict(
                x=0.9,
            )
        ),
    )

#test - labels
labels = go.Scattergeo(
        locations=[abbrev_df.loc[state, 'Postal Code'] for state in states],
        locationmode='USA-states',
        name='',
        mode='text',
        text=[f'<b>{z:.2f}</b>' for z in norm_states_vp],
        textfont=dict(
            family='Courier New',
            size=8,
        )
    )

data = [vp_map, labels]#, dc_overlay, vp_map2]

layout = go.Layout(
    title=go.layout.Title(
        text='Average Voting Power by State',
        yref='paper',
        y=1.0,
        yanchor='bottom',
        pad=dict( # unit: pixels
            b=10,
        ),
    ),
    width=900,
    height=600,
    dragmode=False,
    geo=go.layout.Geo(
        scope='usa',
        projection=go.layout.geo.Projection(type = 'albers usa'),
        showlakes=False,
        lakecolor='rgb(255, 255, 255)'),
)

fig = go.Figure(data=data, layout=layout)
plotly.offline.init_notebook_mode(connected=True)
plotly.offline.iplot(fig)

In [43]:
pio.write_image(fig, 'images/ec_vp_map02.png', format='png', scale=3.0)

In [10]:
# Function to build discretized colormap with user-input # of classes
# Passed in colormap should have only 2 colors in rgb format (defined at 0 and 1)
def create_discrete_cmap(cmap, classes):
    # Get colormap scale values
    beg_val = cmap[0][0]
    end_val = cmap[1][0]
    val_adj = (end_val - beg_val) / float(classes)

    # Extract rgb values into list of ints
    beg_color = [int(x) for x in cmap[0][1].replace(' ', '')[4:-1].split(',')]
    end_color = [int(x) for x in cmap[1][1].replace(' ', '')[4:-1].split(',')]
    color_adj = [(end_color[n] - beg_color[n]) / float(classes-1) for n in range(3)]

    d_cmap = []
    for i in range(classes):
        class_color = 'rgb(' + \
                      ','.join(str(beg_color[n] + i*color_adj[n]) for n in range(3)) + ')'
        d_cmap.append([beg_val + i*val_adj, class_color])
        d_cmap.append([beg_val + (i+1)*val_adj, class_color])
        

    return d_cmap

what = create_discrete_cmap(grayscale, 5)
what

NameError: name 'grayscale' is not defined

In [11]:
# Plot choropleth of racial percentage by state with high/low voting power highlighted

def create_racial_plot(rkey, adj_min=0, adj_max=0):
    most = Pi
    least = G
    grayscale = [
        [0.0, 'rgb(255,255,255)'],
        [1.0, 'rgb(99,99,99)']
    ]

    # Rearrange states so highlighted ones get drawn last
    reorder_idx = []   # for faster insertions, probably should use deque
    for i in range(51):
        if norm_states_vp[i] < .95 or norm_states_vp[i] > 1.5:
            reorder_idx.append(i)
        else:
            reorder_idx.insert(0, i)

    race_keys = [('White', 'HC03_VC99'),
                 ('Hispanic/Latino', 'HC03_VC93'),
                 ('Black/African American', 'HC03_VC100'),
                 ('Asian', 'HC03_VC102')]

    race_percent = [census_df.loc[states[idx], race_keys[rkey][1]] for idx in reorder_idx]
    race = race_keys[rkey][0]

    r_map = go.Choropleth(
        colorscale=grayscale,
        autocolorscale=False,
        #reversescale=True,
        locations=[abbrev_df.loc[states[idx], 'Postal Code'] for idx in reorder_idx], # Needs abbreviations
        z=race_percent,
        locationmode='USA-states',
        marker=dict(
            line=dict(
                color=[most if norm_states_vp[idx] > 1.5 
                       else least if norm_states_vp[idx] < .95 
                       else 'black' for idx in reorder_idx],
                width=[2.5 if norm_states_vp[idx] > 1.5 
                       else 2.5 if norm_states_vp[idx] < .95 
                       else 0.0 for idx in reorder_idx],
            )
        ),
        #zauto=True,
        zmin=min(race_percent) + adj_min,
        zmax=max(race_percent) - adj_max,
        colorbar=dict(
            title='',
            ticksuffix='%',
            thicknessmode='fraction',
            thickness=.030,
            lenmode='fraction',
            len=0.5,
            x=0.85,
            y=0.35,
            outlinewidth=0.5
        ),
        geo='geo',
    )

    most_outline = go.Scattergeo(
        locations=[None],
        locationmode='USA-states',
        name='Highest voting power<br>' + \
             '(> 1.5x average)',
        mode='markers',
        opacity=1.0,
        marker=dict(
            symbol='square-open',
            size=12,
            color=Pi,
            line=dict(
                width=2.5,
            )
        )
    )

    least_outline = go.Scattergeo(
        locations=[None],
        locationmode='USA-states',
        name='Lowest voting power<br>' + \
             '(< 0.95x average)',
        mode='markers',
        marker=dict(
            symbol='square-open',
            size=12,
            color=G,
            line=dict(
                width=2.5,
            )
        )
    )

    
    data = [least_outline, most_outline, r_map]

    layout = go.Layout(
        title=go.layout.Title(
            text='Percent <b>'+race+'</b> Population By State',
            yref='paper',
            y=1.0,
            yanchor='bottom',
            pad=dict( # unit: pixels
                b=20,
            ),
        ),
        width=900,
        height=600,
        dragmode=False,
        geo=go.layout.Geo(
            scope='usa',
            projection=go.layout.geo.Projection(type = 'albers usa'),
            showlakes=False,
            lakecolor='rgb(255, 255, 255)',
        ),
        showlegend=True,
        legend=dict(
            orientation='h',
            traceorder='reversed',
            #tracegroupgap=1000,
            font=dict(
                size=11
            ),
            x=0.5,
            xanchor='center',
            y=1.01,
        ),
    )


    fig = go.Figure(data=data, layout=layout)
    plotly.offline.init_notebook_mode(connected=True)
    plotly.offline.iplot(fig)
    
    return fig
    
# white: 0, adj_min = 10 (Hawaii)
# hispanic: 1
# black: 2, adj_max = 5 (DC)
# asian: 3, adj_max = 20 (Hawaii)
fig = create_racial_plot(0, adj_min=10)

In [62]:
ctest = [
    [0.0, 'white'],
    [0.02, 'white'],
    [0.02, 'blue'],
    [0.49, 'blue'],
    [0.49, 'white'],
    [0.51, 'white'],
    [0.51, 'red'],
    [0.98, 'red'],
    [0.98, 'white'],
    [1.0, 'white']
]

In [122]:
# Plot choropleth of racial percentage by state with high/low voting power highlighted
def create_racial_plot2(rkey, adj_min=0, adj_max=0):
    most = Or #Pi
    least = BG #G
    grayscale = [
        [0.0, 'rgb(255,255,255)'],
        [1.0, 'rgb(99,99,99)']
    ]

    # Rearrange states so highlighted ones get drawn last
    reorder_idx = []   # for faster insertions, probably should use deque
    for i in range(51):
        if norm_states_vp[i] < .95 or norm_states_vp[i] > 1.5:
            reorder_idx.append(i)
        else:
            reorder_idx.insert(0, i)

    race_keys = [('White', 'HC03_VC99'),
                 ('Hispanic/Latino', 'HC03_VC93'),
                 ('Black/African American', 'HC03_VC100'),
                 ('Asian', 'HC03_VC102')]

    race_percent = [census_df.loc[states[idx], race_keys[rkey][1]] for idx in reorder_idx]
    race = race_keys[rkey][0]
    race_min = min(race_percent) + adj_min
    race_max = max(race_percent) - adj_max
    
    plot_w = 900
    plot_h = 600

    r_map = go.Choropleth(
        colorscale=grayscale,
        autocolorscale=False,
        #reversescale=True,
        locations=[abbrev_df.loc[states[idx], 'Postal Code'] for idx in reorder_idx], # Needs abbreviations
        z=race_percent,
        locationmode='USA-states',
        marker=dict(
            line=dict(
                color=[most if norm_states_vp[idx] > 1.5 
                       else least if norm_states_vp[idx] < .95 
                       else 'black' for idx in reorder_idx],
                width=[2.5 if norm_states_vp[idx] > 1.5 
                       else 2.5 if norm_states_vp[idx] < .95 
                       else 0.2 for idx in reorder_idx],
            )
        ),
        #zauto=True,
        zmin=race_min,
        zmax=race_max,
        showscale=False,
        geo='geo',
    )

    most_outline = go.Scattergeo(
        locations=[None],
        locationmode='USA-states',
        name='Highest voting power<br>' + \
             '(> 1.5x average)',
        mode='markers',
        opacity=1.0,
        marker=dict(
            symbol='square-open',
            size=12,
            color=most,
            line=dict(
                width=2.5,
            )
        )
    )

    least_outline = go.Scattergeo(
        locations=[None],
        locationmode='USA-states',
        name='Lowest voting power<br>' + \
             '(< 0.95x average)',
        mode='markers',
        marker=dict(
            symbol='square-open',
            size=12,
            color=least,
            line=dict(
                width=2.5,
            )
        )
    )
    
    bar_pixel = 15
#     horiz_colorbar = go.Scatter(
#         x=list(range(int(race_min), int(race_max), 1)),
#         y=[0]*int(race_max - race_min + 1),
#         showlegend=False,
#         name='what',
#         mode='markers',
#         marker=dict(
#             symbol='square',
#             size=bar_pixel,
#             color=list(range(int(race_min), int(race_max), 1)),
#             colorscale=grayscale,
#             cmin=race_min,
#             cmax=race_max,
#             line=dict(
#                 color=None,
#                 width=0,
#             )
#         )
#     )
    
    cb_res = 100
    cb_step = (race_max - race_min) / float(cb_res)
    horiz_colorbar = go.Scatter(
        x=[cb_step*(x+4) + race_min for x in range(cb_res+1)],
        y=[0]*(cb_res+1),
        showlegend=False,
        name='what',
        mode='markers',
        marker=dict(
            symbol='square',
            size=bar_pixel,
            color=[cb_step*x + race_min for x in range(cb_res+1)],
            colorscale=grayscale,
            cmin=race_min,
            cmax=race_max,
            line=dict(
                color=None,
                width=0,
            )
        )
    )
    
    data = [least_outline, most_outline, r_map, horiz_colorbar]

    layout = go.Layout(
        title=go.layout.Title(
            text='Percent <b>'+race+'</b> Population And Voting Power Extremes',
            yref='paper',
            y=1.0,
            yanchor='bottom',
            pad=dict( # unit: pixels
                b=20,
            ),
        ),
        width=plot_w-160-20,
        height=plot_h-50,
        dragmode=False,
        geo=go.layout.Geo(
            scope='usa',
            projection=go.layout.geo.Projection(type = 'albers usa'),
            showlakes=False,
            lakecolor='rgb(255, 255, 255)',
            domain=dict(
                x=[0,1.0],
                y=[0.1,1.0],
            ),
        ),
        showlegend=True,
        legend=dict(
            orientation='h',
            traceorder='reversed',
            tracegroupgap=10,
            font=dict(
                size=11
            ),
            x=0.525,
            xanchor='right',
            y=0.06,
            yanchor='top'
        ),
        
        margin=dict(
            l=0,
            r=0,
            t=70,
            b=60,
            autoexpand=False,
        ),
        annotations=[dict(
            text='04/22/2019<br>@jeremy_m_joseph',
            font=dict(
                family='Andale Mono',
                size=10,
                color='lightgray',
            ),
            x=.9,
            y=-.13,
            xref='paper',
            yref='paper',
            align='right',
            xanchor='right',
            yanchor='bottom',
            showarrow=False,
        ),]
    )
    
    # Add horizontal colorbar
    layout.update(dict(
        xaxis=dict(
            domain=[0.575, .875],
            visible=True,
            range=[race_min + (cb_step/4.0),
                   race_max + (cb_step/2.0)],
            #range=[race_min-3.25, race_max+1],
            #range=[race_min-2, race_max],
            ticksuffix='%',
            showticksuffix='all',
            tickfont=dict(
                size=11,
            ),
            tickmode='linear',
            dtick=10,
            ticks='',
            showline=False,
            showgrid=False,
            zeroline=False,
            color='black'
        ),
        yaxis=dict(
            domain=[0.0, 0.04],
            visible=False,
            range=[-1, 1],
            showline=False,
            showgrid=False,
            zeroline=False,
        ),
        plot_bgcolor='black',
        shapes=[
            dict(
                type='rect',
                layer='above',
                xref='x',
                xsizemode='scaled',
                yref='y',
                ysizemode='scaled',
                x0=race_min - cb_step, #replace
                x1=race_min,
                y0=-1,
                y1=1,
                fillcolor='black',
                line=dict(
                    width=0,
                )
            ),
            dict(
                type='rect',
                layer='above',
                xref='x',
                xsizemode='scaled',
                yref='y',
                ysizemode='scaled',
                x0=race_max,
                x1=race_max + cb_step,
                y0=-1,
                y1=1,
                fillcolor='black',
                line=dict(
                    width=0,
                ),
            )]
    ))


    fig = go.Figure(data=data, layout=layout)
    plotly.offline.init_notebook_mode(connected=True)
    plotly.offline.iplot(fig)
    
    return fig
    
# white: 0, adj_min = 10 (Hawaii)
# hispanic: 1
# black: 2, adj_max = 5 (DC)
# asian: 3, adj_max = 20 (Hawaii)
fig = create_racial_plot2(0, adj_min=10)

In [123]:
pio.write_image(fig, 'images/mock_bottom_margin7_bold2.png', format='png', scale=3.0)

In [379]:
create_racial_plot(1)

In [404]:
pio.write_image(fig, 'images/legend_bottom.png', format='png', scale=3.0)

In [22]:
fig=create_racial_plot2(2, adj_max=5)

In [381]:
create_racial_plot(3, adj_max=20)